In [89]:
import pandas as pd
df = pd.DataFrame()

# A pandas dataframe will be used to build a file and folder catalog
def add_item_to_catalog(path, type, size=0):
    new_row = pd.DataFrame({'path': path, 'type': type, 'size': size}, index=[0])
    return pd.concat([df, new_row]).reset_index(drop=True)

# Open input file
with open('input.txt') as f:
    lines = f.readlines()

# Build file and folder catalog by decoding each line of the file
for l in lines:
    items = l.strip().split(' ')
    if (items[0] == '$') & (items[1] == 'cd'):
        if items[2] == '/':
            current_path = items[2]
            df=add_item_to_catalog(path=f'{current_path}', type='folder')
        elif items[2] == '..':
            current_path = '/'.join(current_path.split('/')[:-2])+'/'
        else:
            current_path = f'{current_path}{items[2]}/'
    elif items[0] != '$':
        if items[0] == 'dir':
            df=add_item_to_catalog(path=f'{current_path}{items[1]}/', type='folder')
        else:
            df=add_item_to_catalog(path=f'{current_path}{items[1]}', type='file', size=items[0])

# Calculate folder sizes
df['size'] = df['size'].astype('int')
for index, row in df[df['type']=='folder'].iterrows():
    size = df[df['path'].str.startswith(row['path']) & (df['type']=='file')]['size'].sum().item()
    df.loc[df['path']==row['path'], 'size'] = size

# Display catalog
df.sort_values('path')

,path,type,size
0,/,folder,46233734
1,/dpbwg/,folder,308482
14,/dpbwg/dgh/,folder,197049
17,/dpbwg/dgh/lhjmzsl.hzj,file,197049
15,/dpbwg/dpbwg,file,100731
...,...,...,...
13,/sqs/,folder,252966
492,/sqs/zms/,folder,252966
493,/sqs/zms/cvtqph.wwp,file,152096
494,/sqs/zms/mmwzg/,folder,100870


In [90]:
total_size_of_smallest_folders = df[(df['type']=='folder') & (df['size']<= 100000)]['size'].sum()
print('sum of folder sizes below 100000 = {total_size_of_smallest_folders}')

sum of folder sizes below 100000 = {total_size_of_smallest_folders}


In [91]:
total_used_space    = df[df['path']=='/']['size'].sum().item()
current_free_space  = 70000000 - total_used_space
targeted_free_space = 30000000

to_be_dropped = targeted_free_space - current_free_space
print(f'minimum folder size to be dropped = {to_be_dropped}')

size_of_folder_with_min_size = df[(df['type']=='folder') & (df['size']>= to_be_dropped)].sort_values('size').head(1)['size'].sum().item()
print(f'size of existing folder with min size = {size_of_folder_with_min_size}')

minimum folder size to be dropped = 6233734
size of existing folder with min size = 6296435
